---

Tutorial na podstawie:
- http://docs.julialang.org/en/stable/manual
- materialow Piotra Gawrona (Instytut Informatyki Teoretycznej i Stosowanej PAN)

Zawartosc:

- Typy
- Metody
- Konstruktory
- Konwersja i promocja typów
- Moduły
- Obsługa błędów

---

In [1]:
versioninfo()

Julia Version 0.5.0
Commit 3c9d753 (2016-09-19 18:14 UTC)
Platform Info:
  System: Linux (x86_64-linux-gnu)
  CPU: Intel(R) Core(TM) i7 CPU       M 620  @ 2.67GHz
  WORD_SIZE: 64
  BLAS: libopenblas (NO_LAPACKE DYNAMIC_ARCH NO_AFFINITY Nehalem)
  LAPACK: liblapack.so.3
  LIBM: libopenlibm
  LLVM: libLLVM-3.7.1 (ORCJIT, westmere)


## Typy
---
### Wstęp

- System typów w Julii jest dynamiczny, ale ma pewne cechy systemu statycznego.
- Można pisać programy w Julii nie zwracając zbytnio uwagi na typy.
- Typy dzielimy na:
  + **abstrakcyjne** - nie można tworzyć ich instancji,
  + i **konkretne** - nie można po nich dziedziczyć.
 
- Wszystkie byty w Julii są obiektami, które mają dobrze zdefiniowany typ.
- Typ jest znany podczas wykonania a nie kompilacji.
- Tylko wartości mają typy. Zmienne **NIE** mają typów. Zmienne to nazwy związane z wartościami.
- Typy mogą być parametryzowane innymi typami.

### Deklaracja typów
- Do anotowania typów używamy operatora `::`. Ten operator oznacza asercję, mówiącą, że wartość jest danego typu.

In [2]:
# Wyrażenie po lewej jest typu Int
(1+2)::Int

3

In [3]:
# Wyrażenie po lewej jest typu AbstractFloat?
(1+2)::AbstractFloat

LoadError: TypeError: typeassert: expected AbstractFloat, got Int64

Jeżeli doczepimy operator `::` do zmiennej, to deklarujemy, że życzymy sobie by zmienna była skonwertowana do danego typu.

In [4]:
let
    x::Int8 = 10
    typeof(x)
end

Int8

In [5]:
let
    x = 10
    typeof(x)
end

Int64

Takie pseudo-deklaracje zmiennych możemy używać w następujących wersjach
```julia
local x::Int8  #  declaracja zmiennej lokalnej
x::Int8 = 10   # przypisanie 
```
Takie deklaracje dotyczą całego zakresu zmiennej. **Nie działają one w zakresie globalnym**.

In [6]:
let 
    x::Int8=1
    x
end

1

In [7]:
function foo11()
    local y::Int64 
    y = 10
    typeof(y)
end
foo11()

Int64


### Typy abstrakcyjne

Nie mogą być instancjonowane. Służą one jako węzły w grafie typów.

Typy abstrakcyjne definiuje się w następujący sposób.

In [8]:
abstract MyType
abstract MySubType <: MyType
supertype(MyType) |> println
supertype(MySubType) |> println

Any
MyType


Typ `Any` jest super-typem wszystkich typów. Również swoim własnym.

In [9]:
supertype(Any)

Any

Tak możemy sprawdzić typy pochodne po danym.

In [10]:
subtypes(Integer)

4-element Array{Any,1}:
 BigInt  
 Bool    
 Signed  
 Unsigned

Hierarchia typów numerycznych abstrakcyjnych w Julii.

```julia
abstract Number
abstract Real     <: Number
abstract AbstractFloat <: Real
abstract Integer  <: Real
abstract Signed   <: Integer
abstract Unsigned <: Integer
```

Napis `A <: B` oznacza `A` jest podtypem `B`.
Służy on do 
- definiowania bezpośredniego typu nadrzędnego
- lub sprawdzania czy typ `B` jest typem nadrzędnym `A`.

In [11]:
Integer <: Number

true

In [12]:
Integer <: AbstractFloat

false

### Typy bitowe

To typy, które składają się z bitów.

Deklaracja typów numerycznych w Julii wygląda tak:

```julia
bitstype 16 Float16 <: AbstractFloat
bitstype 32 Float32 <: AbstractFloat
bitstype 64 Float64 <: AbstractFloat

bitstype 8  Bool    <: Integer
bitstype 32 Char

bitstype 8  Int8     <: Signed
bitstype 8  UInt8    <: Unsigned
bitstype 16 Int16    <: Signed
bitstype 16 UInt16   <: Unsigned
bitstype 32 Int32    <: Signed
bitstype 32 UInt32   <: Unsigned
bitstype 64 Int64    <: Signed
bitstype 64 UInt64   <: Unsigned
bitstype 128 Int128  <: Signed
bitstype 128 UInt128 <: Unsigned
```

- Nazwa typu ma kluczowe znaczenie. 
- Typy `Bool` i `Int8` są reprezentowane tak samo ale nie można rzutować jednego na drugi automatycznie.

### Typy złożone

Typy złożone to typy podobne do struktur w C.

In [13]:
type Foo
    bar     # Domyślnie typ Any
    toto::Int
    baz::Float64
end

In [14]:
foo = Foo("Murphy", 42, 1.17)

Foo("Murphy",42,1.17)

In [15]:
typeof(foo)

Foo

In [16]:
foo2 = Foo((1,1),1,1.0)

Foo((1,1),1,1.0)

In [17]:
typeof(foo2)

Foo

Do każdego typu tworzone są dwa konstruktory domyślne:
1. pierwszy akceptuje dowolne argumenty i wywołuje funkcję `convert()` na nich konwertując je do typów pól;
2. drugi akceptuje tylko wartości, które pasują dokładnie do typów pól.

In [18]:
# Typ ASCIIString nie jest konwertowalny do Int64
Foo("Murphy","Fish",1.0)

LoadError: MethodError: Cannot `convert` an object of type String to an object of type Int64
This may have arisen from a call to the constructor Int64(...),
since type constructors fall back to convert methods.

In [19]:
# możemy zobaczyć pola zmiennej
fieldnames(foo)

3-element Array{Symbol,1}:
 :bar 
 :toto
 :baz 

In [20]:
# możemy odczytywać pola obiektu
foo.bar

"Murphy"

In [21]:
foo.toto

42

In [22]:
foo.baz

1.17

In [23]:
# Możemy też zmieniać pola obiektu
foo.bar = 1//2

1//2

In [24]:
foo.toto = 1//2

LoadError: InexactError()

In [25]:
foo.baz = 1//2

1//2

In [26]:
foo

Foo(1//2,42,0.5)

Typy złożone, które nie mają pól sa singletonami.

In [27]:
type OneField
    x::Int8
end

object_id(OneField(1)) |> println
object_id(OneField(1)) |> println

is(OneField(1),OneField(1)) |> println

4601279129371692540
3298241052434509624
false


In [28]:
type NoFields
end

object_id(NoFields()) |> println
object_id(NoFields()) |> println

is(NoFields(),NoFields()) |> println

5622019711067984569
5622019711067984569
true


### Typy złożone niezmiennicze

To typy, których obiektów nie można zmieniać.

In [29]:
immutable MyComplex
    real::Float64
    imag::Float64
end

In [30]:
x = MyComplex(1,2)

MyComplex(1.0,2.0)

In [31]:
x.real = 1

LoadError: type MyComplex is immutable

In [32]:
immutable Pair
    a
    b
end
x = Pair([1,2,3],[4,5,6])

Pair([1,2,3],[4,5,6])

In [33]:
x.a=1

LoadError: type Pair is immutable

In [34]:
# Typy niezmiennicze mogą zawierać pola przypisane do obiektów zmiennych
x.a[1]=42

42

In [35]:
x.a, x.b

([42,2,3],[4,5,6])

In [36]:
x.a = [1,2,3]

LoadError: type Pair is immutable

- Obiekty typów niezmiennicze są przekazywane są przez kopie. 
- Typy zwykłe są przekazywane przez referencję.
- Identyczne obiekty typu niezmienniczego są tymi samymi obiektami.

In [37]:
object_id(MyComplex(1,1)) |> println
object_id(MyComplex(1,1)) |> println

14613698013373727160
14613698013373727160


In [38]:
object_id([1,2]) |> println
object_id([1,2]) |> println

14907365932973112706
3538102050045681257


In [39]:
is("1","1")

false

### Sumy mnogościowe typów

In [40]:
IntOrString = Union{Int,AbstractString}

Union{AbstractString,Int64}

In [41]:
42 :: IntOrString

42

In [42]:
"Hello!" :: IntOrString

"Hello!"

In [43]:
1.0 :: IntOrString

LoadError: TypeError: typeassert: expected Union{AbstractString,Int64}, got Float64

### Typy parametryczne

Wszystkie typy typu `DataType` mogą być mieć parametry przez to tworząc rodzaj meta-typów.

Będziemy mówić o trzech klasach typów parametryczne:
- parametryczne typy złożone,
- parametryczne typy abstrakcyjne,
- parametryczne typy bitowe.

#### Parametryczne typy złożone

In [44]:
type Point{T}
    x::T
    y::T
end

In [45]:
Point{Float64}

Point{Float64}

In [46]:
Point{AbstractString}

Point{AbstractString}

In [47]:
Point{Point{Int}}

Point{Point{Int64}}

In [48]:
# Typ parametryczne też jest porządnym typem
Point

Point{T}

In [49]:
# Jest on typem nadrzędnym wobec wszystkich wszytykich typów 
# konkretnych
Point{Float64} <: Point

true

In [50]:
Point{AbstractString} <: Point

true

In [51]:
# To oczywiście nie ma zbytnio sensu
Float64 <: Point

false

In [52]:
Float64 <: Point{Float64}

false

In [53]:
# Typy konkretne o różnych wartościach T 
# nie są wobec siebie typami podrzędnymi
Point{Float64} <: Point{Int64}

false

In [54]:
Point{Int64} <: Point{Float64}

false

In [55]:
# UWAGA!
Float64 <: Real

true

In [56]:
# ALE!
Point{Float64} <: Point{Real}

false

Wynika to z tego, że `Point{Float64}` to po prostu dwie wartości `Float64` jedna po drugiej, a `Point{Real}` to para wskaźników. To kwestia wydajności.

Zatem taka metoda nie jest ciekawa:

In [57]:
function norm(p::Point{Real})
   sqrt(p.x^2 + p.y^2)
end

norm (generic function with 1 method)

In [58]:
# Natomiast ta będzie zachować porządnie
function norm{T<:Real}(p::Point{T})
   sqrt(p.x^2 + p.y^2)
end

norm (generic function with 2 methods)

 Obiekt typu `Point{Float64}` możemy skonstruować w następujący sposób:

In [59]:
p = Point{Float64}(1.0,2.0)

Point{Float64}(1.0,2.0)

In [60]:
typeof(p)

Point{Float64}

Dla typów parametrycznych jest tworzony jeden konstruktor domyślny.

Często przy konstrukcji typu parametrycznego złożonego nie trzeba podawać parametru, gdyż może on być wyliczony na podstawie typów wartości zmiennych.

In [61]:
Point(1.0,2.0)

Point{Float64}(1.0,2.0)

In [62]:
Point(1,2)

Point{Int64}(1,2)

In [63]:
# To oczywiście nie działa
Point(1,2.5)
# Ale można jawnie stworzyć odpowiedni konstruktor

LoadError: MethodError: no method matching Point{T}(::Int64, ::Float64)[0m
Closest candidates are:
  Point{T}{T}(::T, [1m[31m::T[0m) at In[44]:2
  Point{T}{T}(::Any) at sysimg.jl:53[0m

#### Parametryczne typy abstrakcyjne

In [64]:
# Tworzenie
abstract Pointy{T}

In [65]:
# Relacja dziedzicznia
Pointy{Int64} <: Pointy

true

In [66]:
# to nie jest prawda
Pointy{Float64} <: Pointy{Real}

false

In [67]:
Pointy{Real} <: Pointy{Float64}

false

In [68]:
# Typy abstrakcyjne służą do tworzenia hierarchii
type Point2{T} <: Pointy{T}
  x::T
  y::T
end
# Nie można zmienić deklaracji typu w REPLu - stąd Point2

In [69]:
# Relacja herarchii jest zachowana
Point2{Float64} <: Pointy{Float64}

true

In [70]:
Point2{Real} <: Pointy{Real}

true

In [71]:
Point2{AbstractString} <: Pointy{AbstractString}

true

In [72]:
# Możemy ograniczyć zakres dopuszczalnych parametrów
abstract PointyReal{T<:Real}

In [73]:
PointyReal{Float64}

PointyReal{Float64}

In [74]:
PointyReal{Real}

PointyReal{Real}

In [75]:
PointyReal{AbstractString}

LoadError: TypeError: PointyReal: in T, expected T<:Real, got Type{AbstractString}

#### Parametryczne typy bitowe

```julia
# 32-bit system:
bitstype 32 Ptr{T}

# 64-bit system:
bitstype 64 Ptr{T}
```

In [76]:
Ptr{Float64} <: Ptr

true

In [77]:
Ptr{Int64} <: Ptr

true

## Metody

- Metody definiują zachowanie funkcji dla danego zestawu typów jej parametrów.
- Wybór metody, która zostanie wywołana dla danych wartości nazywamy wysyłką (?) (_ang. dispatch_).
- Użycie wszystkich typów parametrów funkcji nazywa się wielokrotną wysyłką (??) (_ang. multiple dispatch_). Jest to **najważniejsza** idea stojąca za językiem Julia.

In [78]:
# Funkcja zdefiniowana dla typów jej parametrów
f(x::Float64, y::Float64) = 2x + y

f (generic function with 1 method)

In [79]:
f(2.0, 3.0)

7.0

In [80]:
f(2.0, 3)

LoadError: MethodError: no method matching f(::Float64, ::Int64)[0m
Closest candidates are:
  f(::Float64, [1m[31m::Float64[0m) at In[78]:2[0m

In [81]:
f(Float32(2.0), 3.0)

LoadError: MethodError: no method matching f(::Float32, ::Float64)[0m
Closest candidates are:
  f([1m[31m::Float64[0m, ::Float64) at In[78]:2[0m

In [82]:
f(x::Number, y::Number) = 2000x + y

f (generic function with 2 methods)

In [83]:
f |> methods

# 2 methods for generic function "f":
f(x::Float64, y::Float64) at In[78]:2
f(x::Number, y::Number) at In[82]:1

In [84]:
f(2.0, 3)

4003.0

In [85]:
f(Float32(2.0), 3.0)

4003.0

In [86]:
# metoda, która wyłapuje wszystkie inne typy parametrów
f(x,y) = println("So long and thanks for all the fish")

f (generic function with 3 methods)

In [87]:
f("dolphin")

LoadError: MethodError: no method matching f(::String)[0m
Closest candidates are:
  f(::Any, [1m[31m::Any[0m) at In[86]:2
  f([1m[31m::Float64[0m, [1m[31m::Float64[0m) at In[78]:2
  f([1m[31m::Number[0m, [1m[31m::Number[0m) at In[82]:1[0m

In [88]:
f("dolphin", 42)

So long and thanks for all the fish


In [89]:
f(Float64, f)

So long and thanks for all the fish


In [90]:
which(f, (Float64, Float64))

f(x::Float64, y::Float64) at In[78]:2

In [91]:
which(f, (Any, AbstractString))

f(x, y) at In[86]:2

In [92]:
methods(+)

# 163 methods for generic function "+":
+(x::Bool, z::Complex{Bool}) at complex.jl:136
+(x::Bool, y::Bool) at bool.jl:48
+(x::Bool) at bool.jl:45
+{T<:AbstractFloat}(x::Bool, y::T) at bool.jl:55
+(x::Bool, z::Complex) at complex.jl:143
+(x::Bool, A::AbstractArray{Bool,N<:Any}) at arraymath.jl:91
+(x::Float32, y::Float32) at float.jl:239
+(x::Float64, y::Float64) at float.jl:240
+(z::Complex{Bool}, x::Bool) at complex.jl:137
+(z::Complex{Bool}, x::Real) at complex.jl:151
+(a::Float16, b::Float16) at float16.jl:136
+(x::Char, y::Integer) at char.jl:40
+(c::BigInt, x::BigFloat) at mpfr.jl:240
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) at gmp.jl:298
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt) at gmp.jl:291
+(a::BigInt, b::BigInt, c::BigInt) at gmp.jl:285
+(x::BigInt, y::BigInt) at gmp.jl:255
+(x::BigInt, c::Union{UInt16,UInt32,UInt64,UInt8}) at gmp.jl:310
+(x::BigInt, c::Union{Int16,Int32,Int64,Int8}) at gmp.jl:326
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) at mpfr.jl:388
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) at mpfr.jl:381
+(a::BigFloat, b::BigFloat, c::BigFloat) at mpfr.jl:375
+(x::BigFloat, c::BigInt) at mpfr.jl:236
+(x::BigFloat, y::BigFloat) at mpfr.jl:205
+(x::BigFloat, c::Union{UInt16,UInt32,UInt64,UInt8}) at mpfr.jl:212
+(x::BigFloat, c::Union{Int16,Int32,Int64,Int8}) at mpfr.jl:220
+(x::BigFloat, c::Union{Float16,Float32,Float64}) at mpfr.jl:228
+{T}(B::BitArray{2}, J::UniformScaling{T}) at linalg/uniformscaling.jl:38
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem, b::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem) at pkg/resolve/versionweight.jl:85
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuild, b::Base.Pkg.Resolve.VersionWeights.VWPreBuild) at pkg/resolve/versionweight.jl:131
+(a::Base.Pkg.Resolve.VersionWeights.VersionWeight, b::Base.Pkg.Resolve.VersionWeights.VersionWeight) at pkg/resolve/versionweight.jl:185
+(a::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue, b::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue) at pkg/resolve/fieldvalue.jl:44
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.CompoundPeriod) at dates/periods.jl:314
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.Period) at dates/periods.jl:312
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.TimeType) at dates/periods.jl:359
+(dt::DateTime, z::Base.Dates.Month) at dates/arithmetic.jl:37
+(dt::DateTime, y::Base.Dates.Year) at dates/arithmetic.jl:13
+(x::DateTime, y::Base.Dates.Period) at dates/arithmetic.jl:64
+(x::Date, y::Base.Dates.Day) at dates/arithmetic.jl:62
+(x::Date, y::Base.Dates.Week) at dates/arithmetic.jl:60
+(dt::Date, z::Base.Dates.Month) at dates/arithmetic.jl:43
+(dt::Date, y::Base.Dates.Year) at dates/arithmetic.jl:17
+(y::AbstractFloat, x::Bool) at bool.jl:57
+{T<:Union{Int128,Int16,Int32,Int64,Int8,UInt128,UInt16,UInt32,UInt64,UInt8}}(x::T, y::T) at int.jl:32
+(x::Integer, y::Ptr) at pointer.jl:108
+(z::Complex, w::Complex) at complex.jl:125
+(z::Complex, x::Bool) at complex.jl:144
+(x::Real, z::Complex{Bool}) at complex.jl:150
+(x::Real, z::Complex) at complex.jl:162
+(z::Complex, x::Real) at complex.jl:163
+(x::Rational, y::Rational) at rational.jl:199
+(x::Integer, y::Char) at char.jl:41
+{N}(i::Integer, index::CartesianIndex{N}) at multidimensional.jl:58
+(c::Union{UInt16,UInt32,UInt64,UInt8}, x::BigInt) at gmp.jl:314
+(c::Union{Int16,Int32,Int64,Int8}, x::BigInt) at gmp.jl:327
+(c::Union{UInt16,UInt32,UInt64,UInt8}, x::BigFloat) at mpfr.jl:216
+(c::Union{Int16,Int32,Int64,Int8}, x::BigFloat) at mpfr.jl:224
+(c::Union{Float16,Float32,Float64}, x::BigFloat) at mpfr.jl:232
+(x::Irrational, y::Irrational) at irrationals.jl:88
+(x::Number) at operators.jl:115
+{T<:Number}(x::T, y::T) at promotion.jl:255
+(x::Number, y::Number) at promotion.jl:190
+(r1::OrdinalRange, r2::OrdinalRange) at operators.jl:505
+{T<:AbstractFloat}(r1::FloatRange{T}, r2::FloatRange{T}) at operators.jl:512
+{T<:AbstractFloat}(r1::LinSpace{T}, r2::LinSpace{T}) at operators.jl:531
+(r1::Union{FloatRange,LinSpa

In [93]:
g(x::Float64, y) = 2x + y

g (generic function with 1 method)

In [94]:
g(x, y::Float64) = x + 2000y

g (generic function with 2 methods)

In [95]:
g(2.0, 3)

7.0

In [96]:
g(2, 3.0)

6002.0

In [97]:
g(2.0, 3.0)

LoadError: MethodError: g(::Float64, ::Float64) is ambiguous. Candidates:
  g(x, y::Float64) at In[94]:1
  g(x::Float64, y) at In[93]:1

In [98]:
# Należy unikać takich sytuacji poprzez zdefiniowanie metody
g(x::Float64, y::Float64) = 2x + 2y

g (generic function with 3 methods)

In [99]:
g(2.0, 3.0)

10.0

### Metody parametryczne

In [100]:
# Sprawdzamy czy zmienne są tego samego typu
same_type{T}(x::T, y::T) = true
same_type(x,y) = false

same_type (generic function with 2 methods)

In [101]:
same_type(1, 2)

true

In [102]:
same_type("foo", 2.0)

false

In [103]:
# Ograniczenie zakresu typów parametrów
same_type_numeric{T<:Number}(x::T, y::T) = true
same_type_numeric(x::Number, y::Number) = false

same_type_numeric (generic function with 2 methods)

In [104]:
same_type_numeric(1, 2)

true

In [105]:
same_type_numeric(1, 2.0)

false

In [106]:
same_type_numeric("foo", 2.0)

LoadError: MethodError: no method matching same_type_numeric(::String, ::Float64)[0m
Closest candidates are:
  same_type_numeric{T<:Number}([1m[31m::T<:Number[0m, ::T<:Number) at In[103]:2
  same_type_numeric([1m[31m::Number[0m, ::Number) at In[103]:3[0m

## Konstruktory

In [107]:
type Fooz
  bar
  baz
end

In [108]:
# Konstruktor domyślny
fooz = Fooz(1,2)

Fooz(1,2)

### Konstruktory zewnętrzne
- Mogą tylko wywoływać inne konstruktory.

In [109]:
Fooz(x) = Fooz(x,x)

Fooz

In [110]:
Fooz(1)

Fooz(1,1)

In [111]:
Fooz() = Fooz(0)

Fooz

In [112]:
Fooz()

Fooz(0,0)

### Konstruktory wewnętrzne
- Konstruktory zewnętrzne pozwalają na:
 1. wymuszenie własności typów,
 2. konstrukcję typów, które odnoszą się same do siebie.
- Są one deklarowane wewnątrz bloku typu oraz
- mają dostęp do funkcji `new()`.
- Jeżeli konstruktor wewnętrzny został zdefiniowany, nie zostają utworzone automatycznie konstruktory domyślne.

In [113]:
type OrderedPair
  x::Real
  y::Real

  OrderedPair(x,y) = x > y ? error("out of order") : new(x,y)
end

In [114]:
OrderedPair(1,2)

OrderedPair(1,2)

In [115]:
OrderedPair(2,1)

LoadError: out of order

In [116]:
# Konstruktory domyślne. Te dwa typy są równoważne.
type T1
  x::Int64
end

type T2
  x::Int64
  T2(x) = new(x)
end

## Konwersja i promocja
### Konwersja
W Julii nie ma ukrytej konwersji typów. Konwersja jest jawnie definiowana.

In [117]:
x = 12

12

In [118]:
typeof(x)

Int64

In [119]:
convert(UInt8, x)

0x0c

In [120]:
typeof(ans)

UInt8

In [121]:
convert(AbstractFloat, x)

12.0

In [122]:
typeof(ans)

Float64

In [123]:
# niemożliwa konwersja
convert(AbstractFloat, "foo")

LoadError: MethodError: Cannot `convert` an object of type String to an object of type AbstractFloat
This may have arisen from a call to the constructor AbstractFloat(...),
since type constructors fall back to convert methods.

Definiowanie własnej konwersji
```julia
function convert(::Type{Bool}, x::Real) 
    x==0 ? false : x==1 ? true : throw(InexactError())
end
```


In [124]:
# Przykład użycia
convert(Bool, 1)

true

In [125]:
convert(Bool, 0)

false

In [126]:
convert(Bool, 1im)

LoadError: InexactError()

In [127]:
convert(Bool, 0im)

false

### Promocja

Promocja występuje wtedy, gdy dwie wartości są promowane do typu "wyższego" typu wspólnego.

In [128]:
promote(1, 2.5)

(1.0,2.5)

In [129]:
promote(1, 2.5, 3)

(1.0,2.5,3.0)

In [130]:
promote(2, 3//4)

(2//1,3//4)

In [131]:
promote(1, 2.5, 3, 3//4)

(1.0,2.5,3.0,0.75)

In [132]:
promote(1.5, im)

(1.5 + 0.0im,0.0 + 1.0im)

In [133]:
promote(1 + 2im, 3//4) |> typeof

Tuple{Complex{Rational{Int64}},Complex{Rational{Int64}}}

- Definiowanie własnych promocji 
```julia
promote_rule(::Type{Float64}, ::Type{Float32} ) = Float64
promote_rule(::Type{UInt8}, ::Type{Int8}) = Int
promote_rule(::Type{BigInt}, ::Type{Int8}) = BigInt
```


In [134]:
# Testowanie promocji
promote_type(Int8, UInt16)

Int64

## Moduły
- Pozwalają na organizację kodu np. w biblioteki.

```julia
module MyModule # definicja modułu, uwaga! nie ma wcięcia
using Lib # będziemy używać modułu Lib

using BigLib: thing1, thing2 #to samo co" BigLib.thing1, BigLib.thing2

import Base.show # działa jak using ale dla jednej nazwy
# Jeżeli chcemy dodawać metody do funkcji, to musimy użyć import

importall OtherLib # importuj wszystkie nazwy z modułu

export MyType, foo # deklaracja eksportowanych nazw

type MyType
    x
end

bar(x) = 2x
foo(a::MyType) = bar(a.x) + 1

show(io, a::MyType) = print(io, "MyType $(a.x)")

include("file1.jl") # wklej zawartość pliku tutaj
end

```

Weźmy taki moduł

```julia
module MyModule

export x, y

x() = "x"
y() = "y"
p() = "p"

end
```


| Polecenie 			| W zakresie pojawia się | Można rozszerzać metodami | 
| - | - | - | 
| using MyModule 			| Wszystkie exportowane nazwy (x oraz y), MyModule.x, MyModule.y oraz MyModule.p | MyModule.x, MyModule.y oraz MyModule.p | 
| using MyModule.x, MyModule.p 	| x oraz p |  | 
| using MyModule: x, p | x oraz p |  | 
| import MyModule | MyModule.x, MyModule.y oraz MyModule.p | MyModule.x, MyModule.y oraz MyModule.p | 
| import MyModule.x, MyModule.p | x oraz p | x oraz p | 
| import MyModule: x, p | x oraz p | x oraz p | 
| importall MyModule | All exported names (x oraz y) | 

## Obsługa błędów
### Wyjątki
- Gdy pojawia się błąd zostaje rzucony wyjątek.
- Zdefiniowany jest szereg wyjątków wbudowanych.


In [135]:
# Przykład pojawienia się wyjątku
sqrt(-1)

LoadError: DomainError:
sqrt will only return a complex result if called with a complex argument. Try sqrt(complex(x)).

In [136]:
# Definicja własnego wyjątku
type MyCustomException <: Exception end

In [137]:
# funkcja throw()
f(x) = x>=0 ? exp(-x) : throw(DomainError()) 
# uwaga na nawiasy rzucamy obiektem typu wyjątek, nie typem

f (generic function with 4 methods)

In [138]:
f(1)

0.36787944117144233

In [139]:
f(-1)

LoadError: DomainError:

In [140]:
# Wyjątki mogą mieć parametry
type MyUndefVarError <: Exception
   var::Symbol
end
function Base.showerror(io::IO, e::MyUndefVarError)
    print(io, e.var, " not defined");
end

In [141]:
throw(UndefVarError(:x))

LoadError: UndefVarError: x not defined

### Błędy
Funkcja `error()` wyrzuca `ErrorException` i kończy wykonanie danej funkcji.

In [142]:
# dodatkowe funkcje informacyjne
info("Hi"); 1+1 # nie wyrzuca wyjątku

INFO: Hi


2

In [143]:
warn("Hi!"); 1+1 # nie wyrzuca wyjątku

2

In [144]:
error("Hi!!!"); 1+1 # wyrzuca wyjątek

LoadError: Hi!!!

#### Konstrukcja `try/catch`.

In [145]:
# przykład
f(x) = try
         sqrt(x)
       catch
         sqrt(complex(x, 0))
       end

f (generic function with 4 methods)

In [146]:
f(1)

1.0

In [147]:
f(-1)

0.0 + 1.0im

In [2]:
sqrt_second(x) = try
        sqrt(x[2])
    catch e
    println(e)
        if isa(e, DomainError)
           sqrt(complex(x[2], 0))
        elseif isa(e, BoundsError)
           sqrt(x)
        end
    end

sqrt_second (generic function with 1 method)

In [3]:
sqrt_second([1 4])

2.0

In [4]:
sqrt_second([1 -4])

DomainError()


0.0 + 2.0im

In [5]:
sqrt_second(9)

BoundsError(#undef,#undef)


3.0

In [6]:
sqrt_second(-9)

BoundsError(#undef,#undef)


LoadError: DomainError:

#### Konstrukcja `finally`

```julia
f = open("file")
try
    # operate on file f
finally
    close(f)
end
```